# 01 — Datenladen & Harmonisierung (Rohdaten)

Dieses Notebook lädt die Rohdaten, harmonisiert IDs/Jahre/Spalten und exportiert konsistente Roh-CSV-Dateien. Es werden **keine** Pro‑Kopf‑Berechnungen, Geo‑Merges oder Analysen durchgeführt.

In [7]:
from __future__ import annotations

from pathlib import Path
import csv
import re

import pandas as pd

NOTEBOOK_DIR = Path.cwd().resolve()
REPO_ROOT = NOTEBOOK_DIR.parent
DATA_DIR = REPO_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

RAW_DIR, PROCESSED_DIR

(PosixPath('/Users/felixipfling/Documents/GitHub/codex_start/data/raw'),
 PosixPath('/Users/felixipfling/Documents/GitHub/codex_start/data/processed'))

## Ergänzung aus `entpackung_notebook.ipynb` (Alterskohorten / 65+)

In [8]:
def extract_bevoelkerung_12411_0018(path: str) -> pd.DataFrame:
    """
    Extrahiert aus Tabelle 12411-0018: kreis_id, kreis_name, bev_total_2024,
    bev_18_35, bev_65_plus.
    """
    df = pd.read_csv(path, sep=";", header=None, encoding="cp1252", dtype=str)

    idx_start = df[0].fillna("").str.contains("31.12.2024").idxmax()
    df_data = df.iloc[idx_start:].reset_index(drop=True)

    df_data.columns = [f"col_{i}" for i in range(df_data.shape[1])]
    df_data = df_data.rename(columns={"col_0": "stichtag", "col_1": "kreis_id", "col_2": "kreis_name"})

    for col in df_data.columns[3:]:
        df_data[col] = pd.to_numeric(df_data[col].str.replace(" ", ""), errors="coerce")

    col_offset = 3
    male_18_35 = [col_offset + i for i in range(5, 9)]
    female_18_35 = [col_offset + 17 + i for i in range(5, 9)]
    male_65_plus = [col_offset + i for i in range(15, 17)]
    female_65_plus = [col_offset + 17 + i for i in range(15, 17)]

    def sum_cols(row, cols):
        return row.iloc[cols].sum(skipna=True)

    out_rows = []
    for _, row in df_data.iterrows():
        kreis_id = str(row["kreis_id"]).strip()
        kreis_name = str(row["kreis_name"]).strip()

        total = sum_cols(row, list(range(col_offset, col_offset + 34)))
        row_out = {
            "kreis_id": kreis_id,
            "kreis_name": kreis_name,
            "bev_total_2024": total,
            "m_18_35": sum_cols(row, male_18_35),
            "w_18_35": sum_cols(row, female_18_35),
            "m_65_plus": sum_cols(row, male_65_plus),
            "w_65_plus": sum_cols(row, female_65_plus),
        }
        out_rows.append(row_out)

    df_out = pd.DataFrame(out_rows)
    df_out["bev_18_35"] = df_out["m_18_35"] + df_out["w_18_35"]
    df_out["bev_65_plus"] = df_out["m_65_plus"] + df_out["w_65_plus"]
    return df_out[["kreis_id", "kreis_name", "bev_total_2024", "bev_18_35", "bev_65_plus"]]


def extract_bevoelkerung_altersstruktur_detailliert(path: str) -> pd.DataFrame:
    """
    Extrahiert die Bevölkerungsstruktur (detailliert) aus einer CSV mit doppeltem Header.
    """
    df = pd.read_csv(path, sep=";", header=[0, 1], encoding="cp1252", dtype=str)

    def clean_age_label(age):
        age = age.strip().lower().replace(" ", "_")
        age = age.replace("bis_unter", "-").replace("jahre", "").replace("und_mehr", "plus")
        age = age.replace("unter_", "0-").replace("__", "_").replace("–", "-")
        return age

    new_columns = []
    for gender, age in df.columns:
        if str(gender).startswith("m"):
            prefix = "m"
        elif str(gender).startswith("w"):
            prefix = "w"
        else:
            new_columns.append(str(gender).lower())
            continue
        age_clean = clean_age_label(str(age))
        new_columns.append(f"{prefix}_{age_clean}")

    df.columns = new_columns

    for col in df.columns:
        if col.startswith("m_") or col.startswith("w_"):
            df[col] = pd.to_numeric(df[col].str.replace(" ", ""), errors="coerce")

    df["kreis_id"] = df["kreis_id"].astype(str).str.strip()
    df["kreis_name"] = df["kreis_name"].astype(str).str.strip()

    alters_cols = [col for col in df.columns if col.startswith(("m_", "w_"))]
    df["bev_total_2024"] = df[alters_cols].sum(axis=1)

    return df[["kreis_id", "kreis_name", "bev_total_2024"] + alters_cols]


def build_generation_kohorten(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Fasst Altersstruktur in Kohorten zusammen – getrennt nach Geschlecht.
    """
    def normalize(colname: str) -> str:
        return colname.lower().replace("_", "").replace("-", "").replace("–", "").replace("plus", "75plus")

    def find_cols(sex, labels):
        norm_labels = [lbl.replace("-", "").replace("–", "").replace("_", "").lower() for lbl in labels]
        return [
            col for col in df_raw.columns
            if col.startswith(sex + "_") and any(normalize(col).endswith(lbl) for lbl in norm_labels)
        ]

    kohorten = {
        "u18": ["0-3", "3-6", "6-10", "10-15", "15-18"],
        "18_34": ["18-20", "20-25", "25-30", "30-35"],
        "35_59": ["35-40", "40-45", "45-50", "50-55", "55-60"],
        "60_74": ["60-65", "65-75"],
        "75plus": ["75plus"],
    }

    df_out = df_raw[["kreis_id", "kreis_name", "bev_total_2024"]].copy()

    for label, altersgruppen in kohorten.items():
        m_cols = find_cols("m", altersgruppen)
        w_cols = find_cols("w", altersgruppen)
        df_out[f"m_{label}"] = df_raw[m_cols].sum(axis=1)
        df_out[f"w_{label}"] = df_raw[w_cols].sum(axis=1)

    return df_out


In [9]:
bev_total_path = NOTEBOOK_DIR/"bev_total_2024.csv" 
bev_65plus_path = NOTEBOOK_DIR/"bev_65plus_2024.csv" 
#bev daten quelle https://ergebnisse.zensus2022.de/datenbank/online/statistic/1000A/details

df_bev_detailliert = extract_bevoelkerung_altersstruktur_detailliert(bev_65plus_path)
df_bev_kohorten = build_generation_kohorten(df_bev_detailliert)
df_bev_65plus = extract_bevoelkerung_12411_0018(bev_65plus_path)

df_bev_kohorten["kreis_id"] = df_bev_kohorten["kreis_id"].astype(str).str.strip().str.zfill(5)
df_bev_65plus["kreis_id"] = df_bev_65plus["kreis_id"].astype(str).str.strip().str.zfill(5)

df_bev_2024 = df_bev_kohorten.merge(
    df_bev_65plus[["kreis_id", "bev_18_35", "bev_65_plus"]],
    on="kreis_id",
    how="left"
)
df_bev_2024.to_csv("raw_bev_kohorten_2024.csv", sep=";", index=False)


In [10]:
display(df_bev_2024)

,kreis_id,kreis_name,bev_total_2024,m_u18,w_u18,m_18_34,w_18_34,m_35_59,w_35_59,m_60_74,w_60_74,m_75plus,w_75plus,bev_18_35,bev_65_plus
0,01001,"Flensburg, kreisfreie Stadt",96326.0,7672.0,7301.0,12829.0,12507.0,15389.0,14808.0,7605.0,8259.0,4156.0,5800.0,25336.0,19455.0
1,01002,"Kiel, kreisfreie Stadt",252668.0,18906.0,18355.0,35758.0,35575.0,39717.0,38828.0,19002.0,21492.0,10149.0,14886.0,71333.0,49169.0
2,01003,"Lï¿½beck, kreisfreie Stadt",216889.0,16967.0,15425.0,23429.0,23136.0,34525.0,35977.0,18768.0,21593.0,10620.0,16449.0,46565.0,51120.0
3,01004,"Neumï¿½nster, kreisfreie Stadt",79809.0,6629.0,6134.0,8893.0,7552.0,13298.0,12819.0,7140.0,7876.0,3782.0,5686.0,16445.0,18429.0
4,01051,"Dithmarschen, Landkreis",133460.0,10675.0,10316.0,12312.0,11031.0,21326.0,21271.0,14272.0,15156.0,7253.0,9848.0,23343.0,35084.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,00nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
488,00nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489,00nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490,00nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


vgr daten quelle:https://www.statistikportal.de/de/vgrdl/ergebnisse-kreisebene/einkommen-kreise

In [11]:
from __future__ import annotations

import re
from pathlib import Path
from typing import Iterable, List, Optional

import pandas as pd

# 1) Dateien (deine Liste)
FILES: List[str] = [
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.1.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.3.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.3.1.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.3.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.1.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.1.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.2.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.2.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.2.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.3.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/1.4.3.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.1.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.1.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.3.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.3.1.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.3.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.4-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.4.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.4.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/2.4.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.3.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.3.1.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.3.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.4-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.4.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.4.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/3.4.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/4-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/5-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.3.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.3.1.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.3.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.4-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.4.1-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.4.2-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/6.4.3-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/7-Tabelle 1.csv",
    NOTEBOOK_DIR/"vgrdl_r2b2_bs2024/8-Tabelle 1.csv",
]
YEARS = ["2020", "2021", "2022", "2023"]
def _strip_codes(text: str) -> str:
    # Abschnittsnummern entfernen (1.3.1.1 etc.)
    text = re.sub(r"^\d+(\.\d+)*\s*", "", text)

    # Klassencodes in Klammern entfernen (C), (A_T), etc.
    text = re.sub(r"\([^)]*\)", "", text)

    return text.strip()


def _read_text_lines(path: str) -> List[str]:
    data = Path(path).read_bytes()
    for enc in ("utf-8", "cp1252", "latin1"):
        try:
            return data.decode(enc).splitlines()
        except UnicodeDecodeError:
            continue
    return data.decode("latin1", errors="replace").splitlines()


def _find_header_row(lines: List[str]) -> int:
    for i, line in enumerate(lines[:300]):
        if "Lfd. Nr." in line and "Regional" in line:
            return i
    raise ValueError("Headerzeile (mit 'Lfd. Nr.' und 'Regional') nicht gefunden.")


def _sanitize_colname(s: str) -> str:
    s = s.strip()
    repl = {"ä": "ae", "ö": "oe", "ü": "ue", "Ä": "Ae", "Ö": "Oe", "Ü": "Ue", "ß": "ss"}
    for k, v in repl.items():
        s = s.replace(k, v)
    s = re.sub(r"[^A-Za-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s


def _extract_measure_colname(lines_before_header: List[str]) -> str:
    """
    Baut den Spaltennamen aus ALLEN relevanten Textzeilen vor dem Header.

    Regeln:
    - alle inhaltlichen Zeilen werden verwendet
    - Abschnittsnummern (1.1.2 etc.) werden entfernt
    - Klassencodes in Klammern (A-T), (C) werden entfernt
    - Sonderzeichen werden normalisiert
    - keine inhaltliche Kürzung
    """

    parts: List[str] = []

    for line in lines_before_header:
        l = line.strip()
        if not l:
            continue
        if "Inhaltsverzeichnis" in l:
            continue

        # Abschnittsnummern am Zeilenanfang entfernen
        l = re.sub(r"^\d+(\.\d+)*\s*", "", l)

        # Klassencodes in Klammern entfernen
        l = re.sub(r"\([^)]*\)", "", l)

        l = l.strip()
        if not l:
            continue

        parts.append(l)

    if not parts:
        return "wert"

    # Alles zusammenführen, NUR technisch sanitizen
    full = " ".join(parts)
    return _sanitize_colname(full)

def _read_table(path: str, header_row: int) -> pd.DataFrame:
    df = pd.read_csv(
        path,
        sep=";",
        header=0,
        skiprows=header_row,
        engine="python",
        dtype=str,
    )
    df = df.dropna(axis=1, how="all")
    df.columns = [c.strip() for c in df.columns]
    df = df.loc[:, [c for c in df.columns if c]]
    return df


def _to_float_de(x: str) -> Optional[float]:
    if x is None:
        return None
    s = str(x).strip()
    if not s:
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def _file_code_prefix(path: str) -> str:
    # "6.4.3-Tabelle 1.csv" -> "6_4_3"
    name = Path(path).name
    m = re.match(r"^([0-9]+(?:\.[0-9]+)*)-", name)
    if not m:
        return ""
    return m.group(1).replace(".", "_")


def _ensure_unique_measure_name(measure: str, existing: set[str]) -> str:
    if measure not in existing:
        return measure
    i = 2
    while f"{measure}__{i}" in existing:
        i += 1
    return f"{measure}__{i}"


def parse_one_file(path: str, years: Iterable[str] = YEARS) -> pd.DataFrame:
    lines = _read_text_lines(path)
    header_row = _find_header_row(lines)

    base_measure = _extract_measure_colname(lines[:header_row])

   # prefix = _file_code_prefix(path)
   # measure = f"{prefix}_{base_measure}" if prefix else base_measure
    measure = base_measure

    measure = _sanitize_colname(measure)

    df = _read_table(path, header_row)

    reg_col = None
    for c in df.columns:
        if "Regional" in c:
            reg_col = c
            break
    if reg_col is None:
        raise ValueError(f"Keine Spalte 'Regional...' in {path} gefunden. Spalten: {list(df.columns)}")

    df[reg_col] = df[reg_col].astype(str).str.strip()
    df = df[df[reg_col].str.match(r"^\d{5}$", na=False)].copy()
    df = df.rename(columns={reg_col: "kreis_id"})

    years = [str(y) for y in years]
    missing = [y for y in years if y not in df.columns]
    if missing:
        raise ValueError(f"In {path} fehlen Jahres-Spalten: {missing}")

    df = df[["kreis_id"] + years].copy()

    out = df.melt(id_vars=["kreis_id"], var_name="jahr", value_name=measure)
    out["jahr"] = out["jahr"].astype(int)
    out[measure] = out[measure].map(_to_float_de)
    return out


def build_target_df(paths: Iterable[str], years: Iterable[str] = YEARS) -> pd.DataFrame:
    paths = list(paths)
    if not paths:
        raise ValueError("paths ist leer. Trag Dateipfade in FILES ein oder nutze glob.")

    merged: Optional[pd.DataFrame] = None
    used_cols: set[str] = {"kreis_id", "jahr"}

    for p in paths:
        df_one = parse_one_file(p, years=years)

        measure_cols = [c for c in df_one.columns if c not in ("kreis_id", "jahr")]
        if len(measure_cols) != 1:
            raise ValueError(f"Unerwartete Spalten in {p}: {list(df_one.columns)}")

        measure = measure_cols[0]
        unique_measure = _ensure_unique_measure_name(measure, used_cols)
        if unique_measure != measure:
            df_one = df_one.rename(columns={measure: unique_measure})
        used_cols.add(unique_measure)

        merged = df_one if merged is None else merged.merge(df_one, on=["kreis_id", "jahr"], how="outer")

    assert merged is not None
    merged = merged.sort_values(["kreis_id", "jahr"]).reset_index(drop=True)
    return merged


# 2) Ziel-DataFrame bauen
target_df = build_target_df(FILES, years=YEARS)

# 3) Optional: prüfen und speichern
print(target_df.shape)
print(target_df.head(10))

# Optional speichern:
# target_df.to_parquet("ziel_df.parquet", index=False)
target_df.to_csv("rawDEmNums_df.csv", sep=";", index=False)


(1592, 55)
  kreis_id  jahr  Arbeitnehmerentgelt_insgesamt_Anteil_am_Land_in  \
0    01001  2020                                              4.3   
1    01001  2021                                              4.3   
2    01001  2022                                              4.4   
3    01001  2023                                              4.4   
4    01002  2020                                             14.4   
5    01002  2021                                             14.3   
6    01002  2022                                             14.3   
7    01002  2023                                             14.2   
8    01003  2020                                             10.3   
9    01003  2021                                             10.2   

   Arbeitnehmerentgelt_Land_und_Forstwirtschaft_Fischerei_in_Mio_EUR  \
0                                                1.0                   
1                                                1.0                   
2            

In [12]:
import pandas as pd
from IPython.display import display

def zero_missing_check(df: pd.DataFrame):
    n = len(df)

    rows = []
    for c in df.columns:
        s = df[c]

        # nur numerische Spalten sinnvoll für 0-Check
        if pd.api.types.is_numeric_dtype(s):
            missing = s.isna().sum()
            zeros = (s == 0).sum()
            valid = n - missing - zeros
        else:
            missing = s.isna().sum()
            zeros = None
            valid = n - missing

        rows.append({
            "variable": c,
            "missing_n": missing,
            "missing_pct": missing / n * 100,
            "zeros_n": zeros,
            "zeros_pct": (zeros / n * 100) if zeros is not None else None,
            "valid_n": valid,
            "valid_pct": valid / n * 100
        })

    out = pd.DataFrame(rows)
    out = out.sort_values(["missing_pct", "zeros_pct"], ascending=False)
    return out

# --------------------------------------------------
# ANWENDUNG
# --------------------------------------------------
print("DEBUG: Missing- und Nullwerte")
print("=" * 60)

display(zero_missing_check(target_df))
# alternativ:
# display(zero_missing_check(panel))


DEBUG: Missing- und Nullwerte


,variable,missing_n,missing_pct,zeros_n,zeros_pct,valid_n,valid_pct
10,Arbeitnehmerentgelt_Handel_Verkehr_und_Lagerei...,1592,100.000000,NaN,NaN,0,0.000000
11,Arbeitnehmerentgelt_Information_und_Kommunikat...,1592,100.000000,NaN,NaN,0,0.000000
13,Arbeitnehmerentgelt_Finanz_Versicherungsdienst...,1592,100.000000,NaN,NaN,0,0.000000
14,Arbeitnehmerentgelt_Grundstuecks_und_Wohnungsw...,1592,100.000000,NaN,NaN,0,0.000000
15,Arbeitnehmerentgelt_Unternehmensdienstleistung...,1592,100.000000,NaN,NaN,0,0.000000
17,Arbeitnehmerentgelt_Oeffentliche_Dienstleistun...,1592,100.000000,NaN,NaN,0,0.000000
18,Arbeitnehmerentgelt_Sonstige_Dienstleistungen_...,1592,100.000000,NaN,NaN,0,0.000000
41,Arbeitnehmerentgelt_je_Arbeitnehmerin_bzw_Arbe...,1592,100.000000,NaN,NaN,0,0.000000
42,Bruttoloehne_und_gehaelter_je_Arbeitnehmerin_b...,1592,100.000000,NaN,NaN,0,0.000000
19,Bruttoloehne_und_gehaelter_insgesamt_in_Mio_EUR,1503,94.409548,0.0,0.000000,89,5.590452


## IDs und Jahresangaben harmonisieren

## Exporte der harmonisierten Rohdaten

bev daten quelle https://ergebnisse.zensus2022.de/datenbank/online/statistic/1000A/details

In [14]:
import pandas as pd
import re

def extract_bev_total_long(
    path: str,
    encoding: str = "cp1252",
    year_from: int | None = None,
    year_to: int | None = None
) -> pd.DataFrame:
    """
    Extrahiert Bevölkerungsstand (total) aus deiner bev_total_2024.csv ins LONG-Format:
      kreis_id, kreis_name, jahr, bev_total

    Ausgangsformat (wie bei dir):
      kreis_id;kreis_name;Stichtag;Stichtag;...
      ;;31.12.2020;31.12.2021;...
      01001;Flensburg,...;89934;91113;...

    Wichtige Punkte:
    - Header ist "zweizeilig": 1. Zeile enthält 'Stichtag' mehrfach, 2. Zeile enthält die echten Datumslabels.
    - Wir lesen daher ohne Header ein, bauen die Spaltennamen selbst, und erzeugen dann LONG.
    - 'jahr' wird aus '31.12.YYYY' extrahiert.
    """

    # ------------------------------------------------------------------
    # 1) Roh einlesen OHNE Header, weil die Datei faktisch einen 2-Zeilen-Header hat
    # ------------------------------------------------------------------
    raw = pd.read_csv(path, sep=";", header=None, dtype=str, encoding=encoding)

    # Minimaler Schutz: Datei sollte mind. 3 Zeilen haben (2 Headerzeilen + Daten)
    if raw.shape[0] < 3:
        raise ValueError(f"Zu wenig Zeilen in Datei: {path}")

    # ------------------------------------------------------------------
    # 2) Erste zwei Zeilen auswerten:
    #    - Zeile 0: ["kreis_id","kreis_name","Stichtag","Stichtag",...]
    #    - Zeile 1: ["", "", "31.12.2020","31.12.2021",...]
    # ------------------------------------------------------------------
    header_top = raw.iloc[0].fillna("").astype(str).tolist()
    header_dates = raw.iloc[1].fillna("").astype(str).tolist()

    # Wir erwarten: Spalte 0 = kreis_id, Spalte 1 = kreis_name
    # Ab Spalte 2 kommen die "Stichtag"-Spalten, deren echte Labels in Zeile 1 stehen.
    if len(header_top) < 3:
        raise ValueError("Unerwartetes Format: es gibt keine Stichtag-Spalten.")

    # ------------------------------------------------------------------
    # 3) Spaltennamen bauen:
    #    - col0 -> 'kreis_id', col1 -> 'kreis_name'
    #    - ab col2: Datum aus zweiter Headerzeile, daraus Jahr extrahieren
    # ------------------------------------------------------------------
    new_cols = ["kreis_id", "kreis_name"]

    years_found = []
    for i in range(2, len(header_top)):
        date_label = header_dates[i].strip()

        # Extrahiere Jahr aus "31.12.2020" etc.
        m = re.search(r"(\d{4})", date_label)
        if not m:
            # Falls leer oder nicht parsebar, trotzdem eine eindeutige Spalte vergeben
            # (damit wir später klar Fehler sehen)
            new_cols.append(f"stichtag_{i}")
            continue

        year = m.group(1)
        years_found.append(year)
        new_cols.append(year)  # Wir benutzen direkt das Jahr als Spaltenname

    # Spaltennamen setzen
    raw.columns = new_cols

    # ------------------------------------------------------------------
    # 4) Datenbereich abschneiden (ab Zeile 2)
    # ------------------------------------------------------------------
    df = raw.iloc[2:].copy()

    # Kreis-ID / Name bereinigen
    df["kreis_id"] = df["kreis_id"].astype(str).str.strip()
    df["kreis_name"] = df["kreis_name"].astype(str).str.strip()

    # ------------------------------------------------------------------
    # 5) Falls gewünscht: Jahresbereich einschränken
    # ------------------------------------------------------------------
    year_cols_all = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
    year_cols_all = sorted(year_cols_all)

    if not year_cols_all:
        raise ValueError(
            "Keine Jahres-Spalten gefunden. "
            f"Gefundene Spalten: {list(df.columns)}"
        )

    # Optional filtern
    year_cols = year_cols_all
    if year_from is not None:
        year_cols = [c for c in year_cols if int(c) >= year_from]
    if year_to is not None:
        year_cols = [c for c in year_cols if int(c) <= year_to]

    if not year_cols:
        raise ValueError(
            f"Nach Filter year_from={year_from}, year_to={year_to} bleiben keine Jahres-Spalten übrig. "
            f"Vorhanden: {year_cols_all}"
        )

    # ------------------------------------------------------------------
    # 6) Werte numerisch machen (Bevölkerung)
    # ------------------------------------------------------------------
    for c in year_cols:
        df[c] = (
            df[c].astype(str)
            .str.replace(" ", "", regex=False)
            .str.replace("\u00a0", "", regex=False)
        )
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # ------------------------------------------------------------------
    # 7) Wide -> Long
    # ------------------------------------------------------------------
    df_long = df.melt(
        id_vars=["kreis_id", "kreis_name"],
        value_vars=year_cols,
        var_name="jahr",
        value_name="bev_total"
    )

    # Typen + sortieren
    df_long["jahr"] = pd.to_numeric(df_long["jahr"], errors="coerce").astype("Int64")
    df_long = df_long.sort_values(["kreis_id", "jahr"]).reset_index(drop=True)

    return df_long


# Anwendung:
df_bev_total_long = extract_bev_total_long(
     NOTEBOOK_DIR/"bev_total_2024.csv",
    encoding="cp1252",
    year_from=2020,
    year_to=2024
)

display(df_bev_total_long.head(20))
print("Rows:", len(df_bev_total_long), "Unique kreise:", df_bev_total_long["kreis_id"].nunique())
print("Jahre:", sorted(df_bev_total_long["jahr"].dropna().unique().tolist()))
df_bev_total_long.to_csv("bevTotal.csv", index=False)


,kreis_id,kreis_name,jahr,bev_total
0,01001,"Flensburg, kreisfreie Stadt",2020,89934.0
1,01001,"Flensburg, kreisfreie Stadt",2021,91113.0
2,01001,"Flensburg, kreisfreie Stadt",2022,96217.0
3,01001,"Flensburg, kreisfreie Stadt",2023,96431.0
4,01001,"Flensburg, kreisfreie Stadt",2024,96326.0
5,01002,"Kiel, kreisfreie Stadt",2020,246601.0
6,01002,"Kiel, kreisfreie Stadt",2021,246243.0
7,01002,"Kiel, kreisfreie Stadt",2022,250064.0
8,01002,"Kiel, kreisfreie Stadt",2023,251751.0
9,01002,"Kiel, kreisfreie Stadt",2024,252668.0


Rows: 2400 Unique kreise: 480
Jahre: [2020, 2021, 2022, 2023, 2024]


pkw daten quelle:https://www-genesis.destatis.de/datenbank/online/statistic/46251/table/46251-0021/table-toolbar/search/s/a3JhZnRzdG9mZmFydGVu#filter=JTdCJTIyaGlkZUVtcHR5Q29scyUyMiUzQWZhbHNlJTJDJTIyaGlkZUVtcHR5Um93cyUyMiUzQWZhbHNlJTJDJTIyY2FwdGlvbiUyMiUzQSU1QiU3QiUyMnZhcmlhYmxlSWQlMjIlM0ElMjI0NjI1MSUyMiUyQyUyMmlkJTIyJTNBJTIyZmlsdGVyLjAlMjIlMkMlMjJ2YWx1ZXNJZHMlMjIlM0ElNUIlMjI0NjI1MSUyMiU1RCUyQyUyMmNoaWxkcmVuJTIyJTNBJTVCJTdCJTIydmFyaWFibGVJZCUyMiUzQSUyMlBLVzAwMSUyMiUyQyUyMmlkJTIyJTNBJTIyZmlsdGVyLjAuMCUyMiUyQyUyMnZhbHVlc0lkcyUyMiUzQSU1QiUyMlFNVSUyMiU1RCUyQyUyMmNoaWxkcmVuJTIyJTNBJTVCJTVEJTJDJTIyc2hvd0FzSW50ZXJsaW5lJTIyJTNBZmFsc2UlMkMlMjJpc0hpZGRlbiUyMiUzQWZhbHNlJTJDJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzElMjIlMkMlMjJwb3NzaWJsZVBsYWNlcyUyMiUzQSU1QiU1RCU3RCU1RCUyQyUyMnNob3dBc0ludGVybGluZSUyMiUzQWZhbHNlJTJDJTIyaXNIaWRkZW4lMjIlM0FmYWxzZSUyQyUyMmJsb2NrQ29kZSUyMiUzQSUyMnMxJTIyJTJDJTIycG9zc2libGVQbGFjZXMlMjIlM0ElNUIlNUQlN0QlNUQlMkMlMjJyb3dIZWFkZXIlMjIlM0ElNUIlN0IlMjJ2YXJpYWJsZUlkJTIyJTNBJTIyU1RBRyUyMiUyQyUyMmlkJTIyJTNBJTIycm93VGl0bGUuMCUyMiUyQyUyMnZhbHVlc0lkcyUyMiUzQSU1QiUyMjIwMjAtMDEtMDElMjIlMkMlMjIyMDIxLTAxLTAxJTIyJTJDJTIyMjAyMi0wMS0wMSUyMiUyQyUyMjIwMjMtMDEtMDElMjIlMkMlMjIyMDI0LTAxLTAxJTIyJTJDJTIyMjAyNS0wMS0wMSUyMiU1RCUyQyUyMmNoaWxkcmVuJTIyJTNBJTVCJTdCJTIydmFyaWFibGVJZCUyMiUzQSUyMktSRUlTRSUyMiUyQyUyMmlkJTIyJTNBJTIycm93VGl0bGUuMC4wJTIyJTJDJTIydmFsdWVzSWRzJTIyJTNBJTVCJTIyMDEwMDElMjIlMkMlMjIwMTAwMiUyMiUyQyUyMjAxMDAzJTIyJTJDJTIyMDEwMDQlMjIlMkMlMjIwMTA1MSUyMiUyQyUyMjAxMDUzJTIyJTJDJTIyMDEwNTQlMjIlMkMlMjIwMTA1NSUyMiUyQyUyMjAxMDU2JTIyJTJDJTIyMDEwNTclMjIlMkMlMjIwMTA1OCUyMiUyQyUyMjAxMDU5JTIyJTJDJTIyMDEwNjAlMjIlMkMlMjIwMTA2MSUyMiUyQyUyMjAxMDYyJTIyJTJDJTIyMDIwMDAlMjIlMkMlMjIwMzEwMSUyMiUyQyUyMjAzMTAyJTIyJTJDJTIyMDMxMDMlMjIlMkMlMjIwMzE1MSUyMiUyQyUyMjAzMTUyJTIyJTJDJTIyMDMxNTMlMjIlMkMlMjIwMzE1NCUyMiUyQyUyMjAzMTU1JTIyJTJDJTIyMDMxNTYlMjIlMkMlMjIwMzE1NyUyMiUyQyUyMjAzMTU4JTIyJTJDJTIyMDMxNTklMjIlMkMlMjIwMzIwMSUyMiUyQyUyMjAzMjQxJTIyJTJDJTIyMDMyNTElMjIlMkMlMjIwMzI1MiUyMiUyQyUyMjAzMjUzJTIyJTJDJTIyMDMyNTQlMjIlMkMlMjIwMzI1NSUyMiUyQyUyMjAzMjU2JTIyJTJDJTIyMDMyNTclMjIlMkMlMjIwMzM1MSUyMiUyQyUyMjAzMzUyJTIyJTJDJTIyMDMzNTMlMjIlMkMlMjIwMzM1NCUyMiUyQyUyMjAzMzU1JTIyJTJDJTIyMDMzNTYlMjIlMkMlMjIwMzM1NyUyMiUyQyUyMjAzMzU4JTIyJTJDJTIyMDMzNTklMjIlMkMlMjIwMzM2MCUyMiUyQyUyMjAzMzYxJTIyJTJDJTIyMDM0MDElMjIlMkMlMjIwMzQwMiUyMiUyQyUyMjAzNDAzJTIyJTJDJTIyMDM0MDQlMjIlMkMlMjIwMzQwNSUyMiUyQyUyMjAzNDUxJTIyJTJDJTIyMDM0NTIlMjIlMkMlMjIwMzQ1MyUyMiUyQyUyMjAzNDU0JTIyJTJDJTIyMDM0NTUlMjIlMkMlMjIwMzQ1NiUyMiUyQyUyMjAzNDU3JTIyJTJDJTIyMDM0NTglMjIlMkMlMjIwMzQ1OSUyMiUyQyUyMjAzNDYwJTIyJTJDJTIyMDM0NjElMjIlMkMlMjIwMzQ2MiUyMiUyQyUyMjA0MDExJTIyJTJDJTIyMDQwMTIlMjIlMkMlMjIwNTExMSUyMiUyQyUyMjA1MTEyJTIyJTJDJTIyMDUxMTMlMjIlMkMlMjIwNTExNCUyMiUyQyUyMjA1MTE2JTIyJTJDJTIyMDUxMTclMjIlMkMlMjIwNTExOSUyMiUyQyUyMjA1MTIwJTIyJTJDJTIyMDUxMjIlMjIlMkMlMjIwNTEyNCUyMiUyQyUyMjA1MTU0JTIyJTJDJTIyMDUxNTglMjIlMkMlMjIwNTE2MiUyMiUyQyUyMjA1MTY2JTIyJTJDJTIyMDUxNzAlMjIlMkMlMjIwNTMxMyUyMiUyQyUyMjA1MzE0JTIyJTJDJTIyMDUzMTUlMjIlMkMlMjIwNTMxNiUyMiUyQyUyMjA1MzM0JTIyJTJDJTIyMDUzNTQlMjIlMkMlMjIwNTM1OCUyMiUyQyUyMjA1MzYyJTIyJTJDJTIyMDUzNjYlMjIlMkMlMjIwNTM3MCUyMiUyQyUyMjA1Mzc0JTIyJTJDJTIyMDUzNzglMjIlMkMlMjIwNTM4MiUyMiUyQyUyMjA1NTEyJTIyJTJDJTIyMDU1MTMlMjIlMkMlMjIwNTUxNSUyMiUyQyUyMjA1NTU0JTIyJTJDJTIyMDU1NTglMjIlMkMlMjIwNTU2MiUyMiUyQyUyMjA1NTY2JTIyJTJDJTIyMDU1NzAlMjIlMkMlMjIwNTcxMSUyMiUyQyUyMjA1NzU0JTIyJTJDJTIyMDU3NTglMjIlMkMlMjIwNTc2MiUyMiUyQyUyMjA1NzY2JTIyJTJDJTIyMDU3NzAlMjIlMkMlMjIwNTc3NCUyMiUyQyUyMjA1OTExJTIyJTJDJTIyMDU5MTMlMjIlMkMlMjIwNTkxNCUyMiUyQyUyMjA1OTE1JTIyJTJDJTIyMDU5MTYlMjIlMkMlMjIwNTk1NCUyMiUyQyUyMjA1OTU4JTIyJTJDJTIyMDU5NjIlMjIlMkMlMjIwNTk2NiUyMiUyQyUyMjA1OTcwJTIyJTJDJTIyMDU5NzQlMjIlMkMlMjIwNTk3OCUyMiUyQyUyMjA2NDExJTIyJTJDJTIyMDY0MTIlMjIlMkMlMjIwNjQxMyUyMiUyQyUyMjA2NDE0JTIyJTJDJTIyMDY0MzElMjIlMkMlMjIwNjQzMiUyMiUyQyUyMjA2NDMzJTIyJTJDJTIyMDY0MzQlMjIlMkMlMjIwNjQzNSUyMiUyQyUyMjA2NDM2JTIyJTJDJTIyMDY0MzclMjIlMkMlMjIwNjQzOCUyMiUyQyUyMjA2NDM5JTIyJTJDJTIyMDY0NDAlMjIlMkMlMjIwNjUzMSUyMiUyQyUyMjA2NTMyJTIyJTJDJTIyMDY1MzMlMjIlMkMlMjIwNjUzNCUyMiUyQyUyMjA2NTM1JTIyJTJDJTIyMDY2MTElMjIlMkMlMjIwNjYzMSUyMiUyQyUyMjA2NjMyJTIyJTJDJTIyMDY2MzMlMjIlMkMlMjIwNjYzNCUyMiUyQyUyMjA2NjM1JTIyJTJ


In [17]:
import pandas as pd
import csv
import re

def read_semicolon_ragged(path: str, encoding: str = "cp1252") -> pd.DataFrame:
    """
    Liest eine Semikolon-Datei ein, bei der die Zeilen unterschiedlich viele Felder haben.
    (Typisch: Titelzeilen mit 1 Feld, Tabellenzeilen mit sehr vielen Feldern.)

    Ergebnis: rechteckiger DataFrame (fehlende Felder werden rechts mit None aufgefüllt).
    """
    rows = []
    max_len = 0

    with open(path, "r", encoding=encoding, newline="") as f:
        reader = csv.reader(f, delimiter=";", quotechar='"')
        for row in reader:
            rows.append(row)
            max_len = max(max_len, len(row))

    padded = [r + [None] * (max_len - len(r)) for r in rows]
    return pd.DataFrame(padded)


def extract_pkw_kba_46251_0021(path: str, encoding: str = "cp1252") -> pd.DataFrame:
    """
    Extrahiert aus KBA/Destatis-Tabelle 46251-0021 (Personenkraftwagen Kreise, Stichtag, Kraftstoffarten, Emissionsgruppen)
    eine saubere Kreistabelle mit:

      stichtag, kreis_id, kreis_name,
      pkw_benzin, pkw_diesel, pkw_gas, pkw_elektrisch, pkw_hybrid, pkw_plugin, pkw_sonstige, pkw_gesamt

    Wichtige Details:
    - Die Datei enthält mehrere Kopfzeilen.
    - Es gibt zwei Header-Zeilen, die die Datenspalten definieren:
        (A) Kraftstoffarten-Zeile: Benzin / Diesel / Gas / Elektro / Hybrid (ohne Plug-in) / Plug-in-Hybrid / Sonstige / Insgesamt
        (B) Euro-Klassen-Zeile: Euro 1 ... Euro 6d ... Sonstige ... Insgesamt
    - In den Datenzeilen folgt auf fast jede Zahl ein Marker (z.B. 'e') in einer eigenen Spalte.
      Diese "Marker-Spalten" entfernen wir, indem wir nur die Spalten behalten, für die in (B) ein Euro-Label steht.
    """

    # 1) Roh einlesen, ohne pandas-Parser-Fehler durch ungleich lange Zeilen
    raw = read_semicolon_ragged(path, encoding=encoding)

    # Kleine Helper: String säubern
    def s(x):
        if x is None:
            return ""
        return str(x).strip()

    # 2) Header-Zeilen finden:
    #    - Kraftstoffzeile erkennen wir daran, dass irgendwo "Benzin" drin steht
    #    - Euro-Zeile erkennen wir daran, dass irgendwo "Euro 1" drin steht
    fuel_row_idx = None
    euro_row_idx = None

    for i in range(len(raw)):
        row_join = " ".join([s(v) for v in raw.iloc[i].tolist()])
        if fuel_row_idx is None and "Benzin" in row_join:
            fuel_row_idx = i
        if euro_row_idx is None and "Euro 1" in row_join:
            euro_row_idx = i
        if fuel_row_idx is not None and euro_row_idx is not None:
            break

    if fuel_row_idx is None or euro_row_idx is None:
        raise ValueError("Konnte die beiden Header-Zeilen (Kraftstoffarten / Euro-Klassen) nicht finden.")

    fuel_row = [s(v) for v in raw.iloc[fuel_row_idx].tolist()]
    euro_row = [s(v) for v in raw.iloc[euro_row_idx].tolist()]

    # 3) Datenstart finden:
    #    Die erste Zeile, deren erstes Feld wie ein Datum aussieht (dd.mm.yyyy), ist der Start eines Stichtag-Blocks.
    date_re = re.compile(r"^\d{2}\.\d{2}\.\d{4}$")
    first_date_idx = None
    for i in range(euro_row_idx + 1, len(raw)):
        if date_re.match(s(raw.iat[i, 0])):
            first_date_idx = i
            break

    if first_date_idx is None:
        raise ValueError("Konnte keine Stichtag-Zeile (dd.mm.yyyy) finden.")

    # 4) Jetzt bauen wir die "wertspalten" aus den zwei Header-Zeilen.
    #    Idee:
    #    - In der Euro-Zeile stehen Euro-Labels (Euro 1, ..., Insgesamt) nur in den echten Werte-Spalten.
    #      Dazwischen sind leere Felder, die zu den Marker-Spalten gehören.
    #    - Also: value_positions = alle Spaltenindizes, wo euro_row[i] nicht leer ist
    value_positions = [j for j, lab in enumerate(euro_row) if lab != ""]

    # Sicherheitscheck: In einem KBA-Kreis-CSV sollten die ersten beiden Spalten die IDs sein.
    # In deinen Zeilen sieht man: 01001;Flensburg,...;322;e;1713;e;...
    # Das heißt: Spalte 0 = kreis_id oder Stichtag, Spalte 1 = kreis_name (wenn kreis-Zeile),
    # und ab Spalte 2 beginnt der Datenbereich (mit Zahlen und Markern).
    # Die Header-Zeilen beginnen aber mit ";;Benzin..." daher sind die Daten ab Index 2 ausgerichtet.
    # Wir kümmern uns gleich beim Extrahieren darum.

    # 5) Aus fuel_row und euro_row bauen wir für jede value_position einen (fuel, euro)-Key.
    #    fuel_row hat den Kraftstoffnamen oft nur am Blockanfang -> wir forward-fillen.
    fuel_by_pos = {}
    current_fuel = ""
    for j in range(len(fuel_row)):
        lab = fuel_row[j]
        if lab != "":
            current_fuel = lab
        fuel_by_pos[j] = current_fuel

    # 6) Normalisierung der Kraftstoffnamen auf kurze, stabile Keys
    def norm_fuel(name: str) -> str:
        n = name.lower().strip()
        n = n.replace("(", "").replace(")", "")
        n = n.replace("ohne plug-in", "").strip()
        n = n.replace("kraftstoffarten", "").strip()
        n = n.replace(" ", "_").replace("-", "_")
        # gezielte Map
        if "benzin" in n:
            return "benzin"
        if "diesel" in n:
            return "diesel"
        if n == "gas" or "gas" in n:
            return "gas"
        if "elektro" in n:
            return "elektrisch"
        if "plug" in n:
            return "plugin"
        if "hybrid" in n:
            return "hybrid"
        if "sonstige" in n:
            return "sonstige"
        if "insgesamt" in n:
            return "gesamt"
        return n if n else "unknown"

    def norm_euro(name: str) -> str:
        n = name.lower().strip()
        n = n.replace(" ", "_").replace("-", "_").replace("(", "").replace(")", "")
        n = n.replace("ohne_6d_und_6d_temp", "ohne_6d")  # optional, nur um zu kürzen
        n = n.replace("6d_temp", "6d_temp")
        if "insgesamt" in n:
            return "insgesamt"
        if "sonstige" in n:
            return "sonstige"
        if "euro" in n:
            return n  # euro_1, euro_2, ...
        return n

    col_keys = []
    for j in value_positions:
        fuel = norm_fuel(fuel_by_pos.get(j, ""))
        euro = norm_euro(euro_row[j])
        col_keys.append((j, fuel, euro))

    # 7) Datenzeilen ab first_date_idx parsen:
    #    - Stichtag-Zeilen: erste Spalte ist ein Datum, der Rest ist leer -> merken wir als current_date
    #    - Kreis-Zeilen: erste Spalte = kreis_id (5-stellig), zweite = kreis_name, dann Zahlen+Marker
    out_rows = []
    current_date = None

    for i in range(first_date_idx, len(raw)):
        c0 = s(raw.iat[i, 0])

        # 7a) Stichtag-Zeile?
        if date_re.match(c0) and s(raw.iat[i, 1]) == "":
            current_date = c0
            continue

        # 7b) Kreis-Zeile? -> kreis_id ist 5-stellig numerisch
        kreis_id = c0
        kreis_name = s(raw.iat[i, 1])

        if not (kreis_id.isdigit() and len(kreis_id) == 5):
            # Alles andere (Leerzeilen, Quellen, Fußnoten) ignorieren
            continue

        # 7c) Werte aus den value_positions ziehen und zu Zahlen konvertieren
        row_dict = {
            "stichtag": current_date,
            "kreis_id": kreis_id,
            "kreis_name": kreis_name,
        }

        for (j, fuel, euro) in col_keys:
            raw_val = s(raw.iat[i, j])

            # fehlende/sonderwerte behandeln: '-' oder '' => NaN
            if raw_val in ("", "-", "nan", "NaN", "None"):
                val = None
            else:
                # Zahlen haben oft Leerzeichen als Tausendertrenner
                raw_val = raw_val.replace(" ", "")
                # rein numerisch machen
                val = pd.to_numeric(raw_val, errors="coerce")

            row_dict[f"{fuel}__{euro}"] = val

        out_rows.append(row_dict)

    df = pd.DataFrame(out_rows)

    # 8) Jetzt die gewünschten Total-Spalten pro Kraftstoffart:
    #    Wir nehmen jeweils die Euro-Klasse "insgesamt" innerhalb des Kraftstoff-Blocks.
    def pick_total(df_in: pd.DataFrame, fuel_key: str) -> pd.Series:
        col = f"{fuel_key}__insgesamt"
        if col not in df_in.columns:
            # Fallback: falls "insgesamt" nicht sauber erkannt wurde, versuchen wir per contains
            candidates = [c for c in df_in.columns if c.startswith(fuel_key + "__") and c.endswith("insgesamt")]
            if len(candidates) == 1:
                col = candidates[0]
            else:
                raise KeyError(f"Konnte die Insgesamt-Spalte für '{fuel_key}' nicht eindeutig finden: {candidates}")
        return df_in[col]

    # Kraftstoffarten, die du später als pkw_* brauchst
    df_out = df[["stichtag", "kreis_id", "kreis_name"]].copy()
    df_out["pkw_benzin"]      = pick_total(df, "benzin")
    df_out["pkw_diesel"]      = pick_total(df, "diesel")
    df_out["pkw_gas"]         = pick_total(df, "gas")
    df_out["pkw_elektrisch"]  = pick_total(df, "elektrisch")
    df_out["pkw_hybrid"]      = pick_total(df, "hybrid")
    df_out["pkw_plugin"]      = pick_total(df, "plugin")
    df_out["pkw_sonstige"]    = pick_total(df, "sonstige")

    # Gesamt: In der Datei gibt es einen Block "Insgesamt" mit eigener "Insgesamt"-Spalte.
    # Das ist i.d.R. die verlässlichste Gesamtzahl (statt Summe der Kraftstoffarten).
    df_out["pkw_gesamt"]      = pick_total(df, "gesamt")

    # 9) Optional: Datentypen auf float (oder Int64) konsistent machen
    numeric_cols = [c for c in df_out.columns if c.startswith("pkw_")]
    df_out[numeric_cols] = df_out[numeric_cols].apply(pd.to_numeric, errors="coerce")

    return df_out


# Anwendung:
path =  NOTEBOOK_DIR/"46251-0021_de.csv"
df_pkw_long = extract_pkw_kba_46251_0021(path)

print("Shape:", df_pkw_long.shape)
display(df_pkw_long.head(10))

# df_pkw_2025 = df_pkw_long[df_pkw_long["stichtag"] == "01.01.2025"].copy()
# display(df_pkw_2025.head())
df_pkw_long.to_csv( "Pkw.csv", index=False)



Shape: (2380, 11)


,stichtag,kreis_id,kreis_name,pkw_benzin,pkw_diesel,pkw_gas,pkw_elektrisch,pkw_hybrid,pkw_plugin,pkw_sonstige,pkw_gesamt
0,01.01.2021,01001,"Flensburg, kreisfreie Stadt",27101.0,16459.0,251.0,321.0,613.0,266.0,18.0,45029.0
1,01.01.2021,01002,"Kiel, kreisfreie Stadt",72647.0,35631.0,753.0,782.0,1716.0,655.0,19.0,112203.0
2,01.01.2021,01003,"LÃ¼beck, kreisfreie Stadt",67458.0,30349.0,809.0,608.0,1304.0,602.0,11.0,101141.0
3,01.01.2021,01004,"NeumÃ¼nster, kreisfreie Stadt",27485.0,14217.0,321.0,408.0,580.0,216.0,7.0,43234.0
4,01.01.2021,01051,"Dithmarschen, Landkreis",48973.0,32253.0,515.0,504.0,719.0,204.0,21.0,83189.0
5,01.01.2021,01053,"Herzogtum Lauenburg, Landkreis",76002.0,40011.0,1054.0,795.0,1248.0,520.0,17.0,119647.0
6,01.01.2021,01054,"Nordfriesland, Landkreis",61335.0,41097.0,665.0,1118.0,949.0,471.0,40.0,105675.0
7,01.01.2021,01055,"Ostholstein, Landkreis",84881.0,41190.0,925.0,670.0,1152.0,511.0,18.0,129347.0
8,01.01.2021,01056,"Pinneberg, Landkreis",120754.0,52584.0,1161.0,1327.0,2730.0,1042.0,36.0,179634.0
9,01.01.2021,01057,"PlÃ¶n, Landkreis",51942.0,27495.0,459.0,550.0,786.0,287.0,12.0,81531.0
